Імпортуємо потрібну бібліотеку.

In [ ]:
import pandas as pd 

In [ ]:
def getDataset(path):
    return pd.read_csv(path, sep=',',encoding='cp1252')

dataset = getDataset('FullDataset.csv')

dataset = dataset[dataset["ID"]<50000]
dataset.info()
dataset.head(10)

def isMatch(str,lst):
    for l in lst:
        if(l == str):
            return 1
    return 0
lst1 = ["Cross Country Skiing", "Sailing", "Athletics", "Alpine Skiing", "Luge", "Rowing", "Swimming", "Canoeing", "Cycling", "Freestyle Skiing"]
lst2 = ["Speed Skating", "Weightlifting", "Bobsleigh", "Archery", "Shooting", "Trampolining", "Snowboarding", "Ski Jumping", "Short Track Speed Skating", "Skeleton"]
lst3 = ["Gymnastics", "Diving", "Figure Skating", "Synchronized Swimming", "Rhythmic Gymnastics", "Curling", "Golf"]
lst4 = ["Judo", "Wrestling", "Taekwondo", "Boxing", "Fencing"]
lst5 = ["Biathlon", "Modern Pentathlon", "Nordic Combined", "Triathlon"]
lst6 = ["Basketball","Ice Hockey", "Badminton", "Football", "Equestrianism", "Handball", "Water Polo", "Tennis", "Hockey", "Softball", "Volleyball", "Table Tennis", "Baseball", "Rugby Sevens", "Beach Volleyball"]

for i in range(len(dataset)) :
    if(isMatch(dataset.iloc[i]["Sport"], lst1) == 1):
        dataset.at[i,"Sport"] = 'Cyclic'
    elif(isMatch(dataset.iloc[i]["Sport"], lst2) == 1):
        dataset.at[i,"Sport"] = 'Speed'
    elif(isMatch(dataset.iloc[i]["Sport"], lst3) == 1):
        dataset.at[i,"Sport"] = 'Coordination'
    elif(isMatch(dataset.iloc[i]["Sport"], lst4) == 1):
        dataset.at[i,"Sport"] = 'Fight'
    elif(isMatch(dataset.iloc[i]["Sport"], lst5) == 1):
        dataset.at[i,"Sport"] = 'All-around'
    else:
        dataset.at[i,"Sport"] = 'Games'


dataset.to_csv("check1.csv", sep=',', encoding='utf-8')

Приберемо непотрібні для нашого аналізу стовпчики.

In [ ]:
dataset = dataset.drop("Unnamed: 0", axis = 1)
dataset = dataset.drop("Code", axis = 1)
dataset = dataset.drop("Year", axis = 1)
dataset = dataset.drop("Team", axis = 1)
dataset = dataset.drop("ID", axis = 1)
dataset.info()
dataset.head(10)

Змінимо значення поля з медалями як булеве чи здобув спортсмен медаль.

In [ ]:
dataset["Medal"].notnull().astype('int')
dataset["Medal"].fillna(0,inplace = True)
dataset["Medal"].fillna(0,inplace = True)

dataset["Medal"]=dataset["Medal"].replace(regex=["Gold"],value='1')
dataset["Medal"]=dataset["Medal"].replace(regex=["Bronze"],value='1')
dataset["Medal"]=dataset["Medal"].replace(regex=["Silver"],value='1')
dataset.to_csv("check1.csv", sep=',', encoding='utf-8')



Ми готові працювати з даними. Тепер давайте розподілимо дані на тренувальні та дані для тесту.

In [ ]:
from sklearn.model_selection import train_test_split
trainingData, testingData  = train_test_split(dataset,test_size=0.30,random_state = 1)

features = pd.concat([trainingData,testingData]).reset_index(drop = True)
features = pd.get_dummies(features)
features = features.drop("Medal_0", axis = 1)
features.rename(columns = {'Medal_1':'Medal'}, inplace = True)
trainingData = features.iloc[:trainingData.shape[0],:]
testingData = features.iloc[trainingData.shape[0]:,:]
print(trainingData.head(10))


Тепер визначимо змінні X та Y для тренувальних та тестувальних даних.

In [ ]:
trainingX  = trainingData.drop(columns='Medal')
trainingY = trainingData['Medal']

testingX  = testingData.drop(columns='Medal')
testingY = testingData['Medal']

trainingY.info()

Додамо необхідні бібліотеки для побудови моделей, побудуємо їх та перевіримо на тестових даних. Спочатку класифікатор Decision Tree.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
decisionTree = DecisionTreeClassifier(max_depth=15,random_state=1)
tree_scores = cross_val_score(decisionTree,trainingX,trainingY,cv = 5)
print('Cvs in Decision Tree:', tree_scores.mean())
decisionTree.fit(trainingX,trainingY)
print('Score of Decision Tree:',decisionTree.score(testingX,testingY)) 

plt.figure(figsize=(30,30))  
tree.plot_tree(decisionTree)
plt.show()
count = 0
arr = decisionTree.predict(testingX)
for i in range(len(arr)):
    print(arr[i])
    if(arr[i] == 1):
        count += 1 
print("Check",count)

Тепер класифікатор Random Forest. 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
randomForest = RandomForestClassifier(max_depth=15)
tree_scores = cross_val_score(randomForest,trainingX,trainingY,cv = 5)
print('Cvs in Random Forest:', tree_scores.mean())
randomForest.fit(trainingX,trainingY)
print('Score of Random Forest:',randomForest.score(testingX,testingY)) 
count = 0
arr = randomForest.predict(testingX)
for i in range(len(arr)):
    print(arr[i])
    if(arr[i] == 1):
        count += 1 
print("Check",count)

Тепер класифікатор Gradient Boosting.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gradientBoosting = GradientBoostingClassifier(learning_rate = 0.1)
tree_scores = cross_val_score(gradientBoosting,trainingX,trainingY,cv = 5)
print('Cvs in Gradient Boosting:', tree_scores.mean())
gradientBoosting.fit(trainingX,trainingY)
print('Score of Gradient Boosting:',gradientBoosting.score(testingX,testingY)) 